# Parallel Computation of Preference Matrices

The notebook below exemplifies how `ir_axioms` can be used to compute preference matrices for all runs submitted to the TREC 2020 Deep Learning passage ranking track.


## Preparation

Install the `ir_axioms` framework and [PyTerrier](https://github.com/terrier-org/pyterrier). In Google Colab, we do this automatically.


In [ ]:
from sys import modules

if "google.colab" in modules:
    !pip install -q ir_axioms[experiments] python-terrier

We initialize PyTerrier and import all required libraries and load the data from [ir_datasets](https://ir-datasets.com/).


In [ ]:
from pyterrier import started, init

if not started():
    init(tqdm="auto")

## Datasets and Index

Using PyTerrier's `get_dataset()`, we load the MS MARCO passage ranking dataset.


In [ ]:
from pyterrier.datasets import get_dataset, Dataset

# Load dataset.
dataset_name = "msmarco-passage/trec-dl-2020/judged"
dataset: Dataset = get_dataset(f"irds:{dataset_name}")

Now define paths where we will store temporary files, datasets, and the search index.


In [ ]:
from pathlib import Path

cache_dir = Path("cache/")
index_dir = cache_dir / "indices" / dataset_name.split("/")[0]

If the index is not ready yet, now is a good time to create it and index the MS MARCO passages.
(Lean back and relax as this may take a while...)


In [ ]:
from pyterrier.index import IterDictIndexer

if not index_dir.exists():
    indexer = IterDictIndexer(str(index_dir.absolute()))
    indexer.index(dataset.get_corpus_iter(), fields=["text"])

## Submitted Runs

Define the path where you have stored the submitted run files.
(You have to manually download the run files from TREC and store them in a directory of your choice. Then adjust the path below.)


In [ ]:
result_dir = Path(
    "/mnt/ceph/storage/data-in-progress/data-research"
    "/web-search/web-search-trec/trec-system-runs"
    "/trec29/deep.passages"
)

We can now load the runs from the run file directory.


In [ ]:
from tqdm.auto import tqdm
from pyterrier.io import read_results

run_files = list(result_dir.iterdir())
run_results = [
    read_results(result_file)
    for result_file in tqdm(
        run_files, desc="Load runs", unit="run", total=len(run_files)
    )
]

Concat the retrieved results from all runs in a single data frame.


In [ ]:
from pandas import concat

all_results = concat(run_results)
all_results

## Import Axioms

Here we're listing which axioms we want to compute preferences for.
Because some axioms require API calls or are computationally expensive, we cache all axioms using `ir_axiom`'s tilde operator (`~`).


In [ ]:
from ir_axioms.axiom import (
    ArgUC,
    QTArg,
    QTPArg,
    aSL,
    PROX1,
    PROX2,
    PROX3,
    PROX4,
    PROX5,
    TFC1,
    TFC3,
    RS_TF,
    RS_TF_IDF,
    RS_BM25,
    RS_PL2,
    RS_QL,
    AND,
    LEN_AND,
    M_AND,
    LEN_M_AND,
    DIV,
    LEN_DIV,
    M_TDC,
    LEN_M_TDC,
    STMC1,
    STMC1_f,
    STMC2,
    STMC2_f,
    LNC1,
    TF_LNC,
    LB1,
    REG,
    ANTI_REG,
    ASPECT_REG,
    REG_f,
    ANTI_REG_f,
    ASPECT_REG_f,
)

axioms = [
    ArgUC(),
    QTArg(),
    QTPArg(),
    aSL(),
    LNC1(),
    TF_LNC(),
    LB1(),
    PROX1(),
    PROX2(),
    PROX3(),
    PROX4(),
    PROX5(),
    REG(),
    REG_f(),
    ANTI_REG(),
    ANTI_REG_f(),
    ASPECT_REG(),
    ASPECT_REG_f(),
    AND(),
    LEN_AND(),
    M_AND(),
    LEN_M_AND(),
    DIV(),
    LEN_DIV(),
    RS_TF(),
    RS_TF_IDF(),
    RS_BM25(),
    RS_PL2(),
    RS_QL(),
    TFC1(),
    TFC3(),
    M_TDC(),
    LEN_M_TDC(),
    STMC1(),
    STMC1_f(),
    STMC2(),
    STMC2_f(),
]
axioms_cached = [~axiom for axiom in axioms]
axiom_names = [axiom.name for axiom in axioms]

## Preference Computation

After having defined the axioms to compute, we create a new PyTerrier pipeline that computes preference matrices for the top-10 results of each system.


In [ ]:
from pyterrier import Transformer
from ir_axioms.backend.pyterrier.transformers import AxiomaticPreferences

compute_preferences = Transformer.from_df(all_results) % 10 >> AxiomaticPreferences(
    axioms=axioms,
    # axioms=axioms_cached,
    axiom_names=axiom_names,
    index=index_dir,
    dataset=dataset_name,
    cache_dir=cache_dir,
    verbose=True,
)

To speed up computation, let's distribute preference matrix computation across 4 cores.


In [ ]:
compute_preferences = compute_preferences.parallel(4)

In the next step, we parallely compute the preference matrices (line 5)
and measure the elapsed time.


In [ ]:
from time import perf_counter_ns

time = perf_counter_ns()

preferences = compute_preferences.transform(dataset.get_topics())

elapsed_time = perf_counter_ns() - time
elapsed_time_seconds = elapsed_time / 1_000_000_000
print(f"Elapsed time: {elapsed_time_seconds:.2f}s")
preferences_per_second = len(preferences) / elapsed_time_seconds
print(f"Preferences per second: {preferences_per_second:.2f}s")

Here's the resulting Pandas `DataFrame` containing all preferences:


In [ ]:
preferences

## Note About Backends

The parallelization is implemented in [PyTerrier](https://pyterrier.readthedocs.io/en/latest/parallel.html).
In this example we use the default [Joblib](https://joblib.readthedocs.io/en/latest/) backend which splits computation per query and runs on multiple cores on the same machine.

However, you could also use the [Ray](https://www.ray.io/) backend.
With Ray you can connect to remote clusters and distribute the workload across multiple machines
(e.g. [Kubernetes](https://docs.ray.io/en/latest/cluster/kubernetes.html), [Hadoop/Spark](https://docs.ray.io/en/latest/cluster/yarn.html), or [Slurm](https://docs.ray.io/en/latest/cluster/slurm.html)).
Please refer to the [Ray documentation](https://docs.ray.io/en/latest/) for detailed instructions on how to connect your cluster.
